In [1]:
import sys
sys.path.insert(0, '../')
import numpy as np
%reload_ext autoreload
%autoreload 1
%aimport allel
%aimport allel.model.ndarray
%aimport allel.model.chunked
%aimport allel.model.dask
allel.__version__

'1.1.0.dev0'

## GenotypeVector

In [2]:
g = allel.GenotypeVector([[0, 1], [-1, -1]]*20)
g.is_phased = np.array([True, False]*20, dtype=bool)
g

0,1,2,3,4,...,35,36,37,38,39
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [3]:
g.display(30, 10)

0,1,2,3,4,5,6,7,8,9,...,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,...,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [4]:
g.displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.,0|1,./.


In [5]:
print(repr(g))

<GenotypeVector shape=(40, 2) dtype=int64>
0|1 ./. 0|1 ./. 0|1 ... ./. 0|1 ./. 0|1 ./.


In [6]:
print(g)

0|1 ./. 0|1 ./. 0|1 ... ./. 0|1 ./. 0|1 ./.


In [7]:
print(g.to_str(40))

0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./. 0|1 ./.


In [8]:
g.concatenate(g)

0,1,2,3,4,...,75,76,77,78,79
0|1,./.,0|1,./.,0|1,...,./.,0|1,./.,0|1,./.


In [9]:
g.concatenate(g, axis=1)

array([[ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1],
       [ 0,  1,  0,  1],
       [-1, -1, -1, -1]])

## GenotypeArray

In [10]:
g = allel.GenotypeArray([[[0, 1], [-1, -1]]*20]*20)
g.is_phased = np.array([[True, False, False, False]*10]*20, dtype=bool)
g.mask = np.array([[1, 0, 0, 0, 0]*8]*20, dtype=bool)

In [11]:
print(g)

.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.



In [12]:
print(repr(g))

<GenotypeArray shape=(20, 40, 2) dtype=int64>
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.



In [13]:
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
.|. ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.

In [14]:
g.display(row_threshold=10, row_edgeitems=5, col_threshold=20, col_edgeitems=10)

<GenotypeArray shape=(20, 40, 2) dtype=int64> 0 1 2 3 4 5 6 7 8 9 ... 30 31 32 33 34 35 36 37 38 39 0 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 1 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 2 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 3 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 4 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. ... ... 15 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 16 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 17 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 18 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./. 19 .|. ./. 0/1 ./. 0|1 ./. 0/1 ./. 0|1 ./. ... ./. ./. 0|1 ./. 0/1 ./. 0|1 ./. 0/1 ./.

In [15]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,.|.,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,./.,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [16]:
g[0]

0,1,2,3,4,...,35,36,37,38,39
.|.,./.,0/1,./.,0|1,...,./.,0|1,./.,0/1,./.


In [17]:
g[:, 4]

0,1,2,3,4,...,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,...,0|1,0|1,0|1,0|1,0|1


In [18]:
g[:, 4].displayall()

0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19
0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1,0|1


In [19]:
g.mask = None
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.

In [20]:
g.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39
0,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
1,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
2,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
3,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
4,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
5,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
6,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
7,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
8,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.
9,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.,0|1,./.,0/1,./.


In [21]:
g.concatenate(g)

<GenotypeArray shape=(40, 40, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.

In [22]:
g.concatenate(g, axis=1)

<GenotypeArray shape=(20, 80, 2) dtype=int64>
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
0|1 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.

In [23]:
g.to_n_alt()

array([[1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,
        1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0],
       [1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 

## HaplotypeArray

In [24]:
h = g.to_haplotypes()

In [25]:
h

<HaplotypeArray shape=(20, 80) dtype=int64>
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
...
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .

In [26]:
print(repr(h))

<HaplotypeArray shape=(20, 80) dtype=int64>
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
...
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .



In [27]:
print(h)

0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
...
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .



In [28]:
h.display(20, 40)

,0,1,2,3,4,...,75,76,77,78,79
0,0,1,.,.,0,...,.,0,1,.,.
1,0,1,.,.,0,...,.,0,1,.,.
2,0,1,.,.,0,...,.,0,1,.,.
3,0,1,.,.,0,...,.,0,1,.,.
4,0,1,.,.,0,...,.,0,1,.,.
5,0,1,.,.,0,...,.,0,1,.,.
6,0,1,.,.,0,...,.,0,1,.,.
7,0,1,.,.,0,...,.,0,1,.,.
8,0,1,.,.,0,...,.,0,1,.,.
9,0,1,.,.,0,...,.,0,1,.,.


In [29]:
h.displayall()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
0,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
1,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
2,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
3,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
4,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
5,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
6,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
7,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
8,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.
9,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.,0,1,.,.


In [30]:
h.concatenate(h)

<HaplotypeArray shape=(40, 80) dtype=int64>
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
...
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .

In [31]:
h.concatenate(h, axis=1)

<HaplotypeArray shape=(20, 160) dtype=int64>
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
...
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .
0 1 . . 0 ... . 0 1 . .

## AlleleCountsArray

In [32]:
ac = g.count_alleles()
ac

<AlleleCountsArray shape=(20, 2) dtype=int32>
20 20
20 20
20 20
...
20 20
20 20
20 20

In [33]:
ac.displayall()

,0,1
0,20,20
1,20,20
2,20,20
3,20,20
4,20,20
5,20,20
6,20,20
7,20,20
8,20,20
9,20,20


In [34]:
print(ac)

20 20
20 20
20 20
...
20 20
20 20
20 20



In [35]:
print(repr(ac))

<AlleleCountsArray shape=(20, 2) dtype=int32>
20 20
20 20
20 20
...
20 20
20 20
20 20



In [36]:
ac + 5

<AlleleCountsArray shape=(20, 2) dtype=int32>
25 25
25 25
25 25
...
25 25
25 25
25 25

In [37]:
ac + ac

<AlleleCountsArray shape=(20, 2) dtype=int32>
40 40
40 40
40 40
...
40 40
40 40
40 40

In [38]:
ac.concatenate(ac)

<AlleleCountsArray shape=(40, 2) dtype=int32>
20 20
20 20
20 20
...
20 20
20 20
20 20

In [39]:
ac.concatenate(ac, axis=1)

array([[20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20],
       [20, 20, 20, 20]], dtype=int32)

## GenotypeAlleleCounts

In [40]:
gac = g.to_allele_counts()
gac

<GenotypeAlleleCountsArray shape=(20, 40, 2) dtype=uint8>
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
...
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0

In [41]:
print(gac)

1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
...
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0



In [42]:
print(repr(gac))

<GenotypeAlleleCountsArray shape=(20, 40, 2) dtype=uint8>
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
...
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0
1:1 0:0 1:1 0:0 1:1 ... 0:0 1:1 0:0 1:1 0:0



In [43]:
g[:, 0] = 2

In [44]:
g.to_allele_counts()

<GenotypeAlleleCountsArray shape=(20, 40, 3) dtype=uint8>
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
...
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0

In [45]:
gac[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
1,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
2,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
3,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0
4,1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0


In [46]:
gac[0]

0,1,2,3,4,...,35,36,37,38,39
1:1,0:0,1:1,0:0,1:1,...,0:0,1:1,0:0,1:1,0:0


In [47]:
gac[:, 0]

0,1,2,3,4,...,15,16,17,18,19
1:1,1:1,1:1,1:1,1:1,...,1:1,1:1,1:1,1:1,1:1


In [48]:
gac[:3].is_called()

array([[ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False],
       [ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False],
       [ True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False,  True, False,  True, False,  True,
        False,  True, False,  True, False,  True, False,  True, False,
         True, False,  True, False]], dtype=bool)

In [49]:
gac[0].is_called()

array([ True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False,  True, False,  True, False,  True,
       False,  True, False,  True, False,  True, False,  True, False,
        True, False,  True, False], dtype=bool)

## VariantTable

In [50]:
pos = np.unique(np.random.randint(0, 1000000, size=50000))
ref = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
alt = np.random.choice([b'A', b'C', b'T', b'G'], size=pos.shape[0])
ra = np.rec.fromarrays([pos, ref, alt], names=['POS', 'REF', 'ALT'])
vtbl = allel.VariantTable(ra)
vtbl

<VariantTable shape=(48772,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(    16, b'G', b'G') (    22, b'C', b'G') (    54, b'G', b'G') ...,
 (999938, b'C', b'C') (999977, b'C', b'C') (999993, b'G', b'G')]

In [51]:
vtbl[:5]

,POS,REF,ALT
0,16,b'G',b'G'
1,22,b'C',b'G'
2,54,b'G',b'G'
3,61,b'G',b'G'
4,94,b'T',b'G'


In [52]:
vtbl[:20].displayall()

,POS,REF,ALT
0,16,b'G',b'G'
1,22,b'C',b'G'
2,54,b'G',b'G'
3,61,b'G',b'G'
4,94,b'T',b'G'
5,109,b'G',b'C'
6,165,b'A',b'A'
7,170,b'G',b'C'
8,174,b'T',b'A'
9,179,b'T',b'G'


In [53]:
vtbl.take(list(range(5)))

,POS,REF,ALT
0,16,b'G',b'G'
1,22,b'C',b'G'
2,54,b'G',b'G'
3,61,b'G',b'G'
4,94,b'T',b'G'


In [54]:
vtbl.compress(np.random.randint(0, 2, size=vtbl.shape[0]).astype(bool))

<VariantTable shape=(24495,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(    22, b'C', b'G') (   109, b'G', b'C') (   179, b'T', b'G') ...,
 (999898, b'G', b'T') (999938, b'C', b'C') (999977, b'C', b'C')]

In [55]:
vtbl.concatenate(vtbl)

<VariantTable shape=(97544,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(    16, b'G', b'G') (    22, b'C', b'G') (    54, b'G', b'G') ...,
 (999938, b'C', b'C') (999977, b'C', b'C') (999993, b'G', b'G')]

In [56]:
vtbl[['POS', 'ALT']]

/home/aliman/pyenv/dev-scikit-allel-py36-20170614/lib/python3.6/site-packages/numpy/core/records.py:507: FutureWarning: Numpy has detected that you may be viewing or writing to an array returned by selecting multiple fields in a structured array. 

This code may break in numpy 1.13 because this will return a view instead of a copy -- see release notes for details.
  return obj.view(dtype=(self.dtype.type, obj.dtype))


<VariantTable shape=(48772,) dtype=(numpy.record, [('POS', '<i8'), ('ALT', 'S1')])>
[(    16, b'G') (    22, b'G') (    54, b'G') ..., (999938, b'C')
 (999977, b'C') (999993, b'G')]

In [57]:
print(vtbl)

[(    16, b'G', b'G') (    22, b'C', b'G') (    54, b'G', b'G') ...,
 (999938, b'C', b'C') (999977, b'C', b'C') (999993, b'G', b'G')]


In [58]:
print(repr(vtbl))

<VariantTable shape=(48772,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(    16, b'G', b'G') (    22, b'C', b'G') (    54, b'G', b'G') ...,
 (999938, b'C', b'C') (999977, b'C', b'C') (999993, b'G', b'G')]


## Chunked stuff

In [59]:
gc = allel.GenotypeChunkedArray(g).copy()
gc

<GenotypeChunkedArray shape=(20, 40, 2) dtype=int64 chunks=(20, 40, 2)
   nbytes=12.5K cbytes=450 cratio=28.4
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [60]:
print(gc)

2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
...
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.



In [61]:
print(repr(gc))

<GenotypeChunkedArray shape=(20, 40, 2) dtype=int64 chunks=(20, 40, 2)
   nbytes=12.5K cbytes=450 cratio=28.4
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>


In [62]:
gc[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
3,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
4,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [63]:
gc[:, :5]

<GenotypeArray shape=(20, 5, 2) dtype=int64>
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
...
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1
2/2 ./. 0/1 ./. 0/1

In [64]:
gc[5]

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [65]:
gc[:, 5]

0,1,2,3,4,...,15,16,17,18,19
./.,./.,./.,./.,./.,...,./.,./.,./.,./.,./.


In [66]:
x = gc.take([0, 2, 4], axis=0)
x

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [67]:
gc.take([0, 1, 2], axis=1)

<GenotypeChunkedArray shape=(20, 3, 2) dtype=int64 chunks=(20, 3, 2)
   nbytes=960 cbytes=386 cratio=2.5
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [68]:
gcna = gc.to_n_alt()
gcna

<ChunkedArrayWrapper shape=(20, 40) dtype=int8 chunks=(20, 40)
   nbytes=800 cbytes=360 cratio=2.2
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [69]:
gcna > 0

<ChunkedArrayWrapper shape=(20, 40) dtype=bool chunks=(20, 40)
   nbytes=800 cbytes=352 cratio=2.3
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [70]:
hc = gc.to_haplotypes()
hc

<HaplotypeChunkedArray shape=(20, 80) dtype=int64 chunks=(20, 80)
   nbytes=12.5K cbytes=428 cratio=29.9
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [71]:
hc.take([0, 1, 2], axis=1)

<HaplotypeChunkedArray shape=(20, 3) dtype=int64 chunks=(20, 3)
   nbytes=480 cbytes=361 cratio=1.3
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [72]:
ac = g.count_alleles()
ac

<AlleleCountsArray shape=(20, 3) dtype=int32>
19 19  2
19 19  2
19 19  2
...
19 19  2
19 19  2
19 19  2

In [73]:
acc = gc.count_alleles()
acc

<AlleleCountsChunkedArray shape=(20, 3) dtype=int32 chunks=(20, 3)
   nbytes=240 cbytes=356 cratio=0.7
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [74]:
gc.is_called()

<ChunkedArrayWrapper shape=(20, 40) dtype=bool chunks=(20, 40)
   nbytes=800 cbytes=352 cratio=2.3
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [75]:
gc.to_n_alt()

<ChunkedArrayWrapper shape=(20, 40) dtype=int8 chunks=(20, 40)
   nbytes=800 cbytes=360 cratio=2.2
   compression=blosc compression_opts={'cname': 'lz4', 'clevel': 5, 'shuffle': 1}
   values=zarr.core.Array>

In [76]:
vctbl = allel.VariantChunkedTable(vtbl).copy()
vctbl

<VariantChunkedTable shape=(48772,) dtype=[('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')]
   nbytes=476.3K cbytes=127.4K cratio=3.7
   values=allel.chunked.storage_zarr.ZarrTable>

In [77]:
vctbl[:]

<VariantTable shape=(48772,) dtype=(numpy.record, [('POS', '<i8'), ('REF', 'S1'), ('ALT', 'S1')])>
[(    16, b'G', b'G') (    22, b'C', b'G') (    54, b'G', b'G') ...,
 (999938, b'C', b'C') (999977, b'C', b'C') (999993, b'G', b'G')]

## Dask arrays

In [78]:
g

<GenotypeArray shape=(20, 40, 2) dtype=int64>
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
...
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.
2|2 ./. 0/1 ./. 0|1 ... ./. 0|1 ./. 0/1 ./.

In [79]:
gd = allel.GenotypeDaskArray(g)

In [80]:
gd.mask is None

True

In [81]:
gd.is_phased is None

True

In [82]:
print(repr(gd))

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>


In [83]:
print(gd)

2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
...
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.



In [84]:
gd

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>

In [85]:
gd.values

dask.array<array, shape=(20, 40, 2), dtype=int64, chunksize=(20, 40, 2)>

In [86]:
gd.values.name

'array-02899a22ff9fd8802550da9027abb49d'

In [87]:
gd[:]

<GenotypeDaskArray shape=(20, 40, 2) dtype=int64>

In [88]:
gd[:5]

,0,1,2,3,4,...,35,36,37,38,39
0,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
1,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
2,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
3,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.
4,2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [89]:
gd.compute()

<GenotypeArray shape=(20, 40, 2) dtype=int64>
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
...
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.
2/2 ./. 0/1 ./. 0/1 ... ./. 0/1 ./. 0/1 ./.

In [90]:
gd.concatenate(gd)

<GenotypeDaskArray shape=(40, 40, 2) dtype=int64>

In [91]:
gd.concatenate(gd, axis=1)

<GenotypeDaskArray shape=(20, 80, 2) dtype=int64>

In [92]:
hd = gd.to_haplotypes()
hd

<HaplotypeDaskArray shape=(20, 80) dtype=int64>

In [93]:
hd.compute()

<HaplotypeArray shape=(20, 80) dtype=int64>
2 2 . . 0 ... . 0 1 . .
2 2 . . 0 ... . 0 1 . .
2 2 . . 0 ... . 0 1 . .
...
2 2 . . 0 ... . 0 1 . .
2 2 . . 0 ... . 0 1 . .
2 2 . . 0 ... . 0 1 . .

In [94]:
acd = gd.count_alleles()
acd

<AlleleCountsDaskArray shape=(20, 3) dtype=int64>

In [95]:
acd.compute()

<AlleleCountsArray shape=(20, 3) dtype=int64>
19 19  2
19 19  2
19 19  2
...
19 19  2
19 19  2
19 19  2

In [96]:
gd[4]

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [97]:
gd[4].compute()

0,1,2,3,4,...,35,36,37,38,39
2/2,./.,0/1,./.,0/1,...,./.,0/1,./.,0/1,./.


In [98]:
acd + acd

<AlleleCountsDaskArray shape=(20, 3) dtype=int64>

In [99]:
(acd + acd).compute()

<AlleleCountsArray shape=(20, 3) dtype=int64>
38 38  4
38 38  4
38 38  4
...
38 38  4
38 38  4
38 38  4

In [100]:
gacd = allel.GenotypeAlleleCountsDaskArray(g.to_allele_counts())
gacd

<GenotypeAlleleCountsDaskArray shape=(20, 40, 3) dtype=uint8>

In [101]:
gacd.compute()

<GenotypeAlleleCountsArray shape=(20, 40, 3) dtype=uint8>
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
...
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0
0:0:2 0:0:0 1:1:0 0:0:0 1:1:0 ... 0:0:0 1:1:0 0:0:0 1:1:0 0:0:0

In [102]:
gacd.count_alleles()

<AlleleCountsDaskArray shape=(20, 3) dtype=uint64>

In [103]:
gacd.count_alleles().compute()

<AlleleCountsArray shape=(20, 3) dtype=uint64>
19 19  2
19 19  2
19 19  2
...
19 19  2
19 19  2
19 19  2